In [35]:

import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
import json # library to handle JSON files
import bs4 as bs
import urllib.request

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

# geocoder
!conda install -c conda-forge geocoder --yes 
import geocoder

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# use the inline backend to generate the plots within the browser
%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('ggplot') # optional: for ggplot-like style

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0
print('Libraries imported.')

'conda' is not recognized as an internal or external command,
operable program or batch file.
'conda' is not recognized as an internal or external command,
operable program or batch file.
'conda' is not recognized as an internal or external command,
operable program or batch file.


Matplotlib version:  3.2.2
Libraries imported.


In [36]:
# Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
source = urllib.request.urlopen(url).read()
soup = bs.BeautifulSoup(source,'html.parser')

table = soup.find('table')
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        l.append(row)

In [37]:
df = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [38]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']
df.head(10)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [39]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.loc[df['Neighbourhood']=='Not assigned', ['Neighbourhood']] = 'Queen\'s Park'
df.head(10)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [40]:

# Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org
url='https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods'
skip_table = 1

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
source = urllib.request.urlopen(url).read()
soup = bs.BeautifulSoup(source,'html.parser')

table = soup.find_all('table')[skip_table]
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    #print (tr)
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        l.append(row)

In [41]:
df1 = pd.DataFrame(l, columns=["Neighbourhood", "FM", "Census", "Population", "Land Area", "Density", "Population %", "Income", "Commuting", "Renters", "2nd Language", "2nd Language %"])

# clean up the data and dropping unwanted columns
df1 = df1[df1.Neighbourhood != 'Toronto CMA Average']
df1 = df1.drop('FM', 1)
df1 = df1.drop('Census', 1)
df1 = df1.drop('Renters', 1)

# change to the proper datatype
df1['Population'] = df1['Population'].str.replace(',','')
df1['Population'] = df1['Population'].apply(pd.to_numeric)
#print (df1.dtypes)

# display sample data
df1.head(10)

,Neighbourhood,Population,Land Area,Density,Population %,Income,Commuting,2nd Language,2nd Language %
1,Agincourt,44577,12.45,3580,4.6,"25,750",11.1,Cantonese (19.3%),19.3% Cantonese
2,Alderwood,11656,4.94,2360,-4.0,"35,239",8.8,Polish (6.2%),06.2% Polish
3,Alexandra Park,4355,0.32,"13,609",0.0,"19,687",13.8,Cantonese (17.9%),17.9% Cantonese
4,Allenby,2513,0.58,4333,-1.0,"245,592",5.2,Russian (1.4%),01.4% Russian
5,Amesbury,17318,3.51,"4,934",1.1,"27,546",16.4,Spanish (6.1%),06.1% Spanish
6,Armour Heights,4384,2.29,1914,2.0,"116,651",10.8,Russian (9.4%),09.4% Russian
7,Banbury,6641,2.72,2442,5.0,"92,319",6.1,Unspecified Chinese (5.1%),05.1% Unspecified Chinese
8,Bathurst Manor,14945,4.69,3187,12.3,"34,169",13.4,Russian (9.5%),09.5% Russian
9,Bay Street Corridor,4787,0.11,"43,518",3.0,"40,598",17.1,Mandarin (9.6%),09.6% Mandarin
10,Bayview Village,12280,4.14,"2,966",41.6,"46,752",14.4,Cantonese (8.4%),08.4% Cantonese


In [42]:
# get the data size
df.shape
df1.shape

(174, 9)

In [ ]:
df1['Latitude'] = '43.638093'
df1['Longitude'] = '-79.4665843'

# dummy values in case that geocoder is not working for the day
#df1['Latitude'] = 43.638093
#df1['Longitude'] = -79.4665843

In [ ]:
# loop through the postal code to find out the latitude and longitude
for index, data in df1.iterrows():
    lat_lng_coords = None
    #print ('Now at PostalCode:', data['PostalCode'])
    #print ('Now at Neighbourhood:', data['Neighbourhood'])
    while(lat_lng_coords is None):
        #g = geocoder.google('{}, Toronto, Ontario'.format(data['PostalCode']))
        g = geocoder.google('{}, Toronto, Ontario'.format(data['Neighbourhood']))
        lat_lng_coords = g.latlng
    data['Latitude'] = lat_lng_coords[0]
    data['Longitude'] = lat_lng_coords[1]
    print ('Neighbourhood:', data['Neighbourhood'], 'Latitude:', data['Latitude'], 'Longitude:', data['Longitude'])
    
# loop through the postal code to find out the latitude and longitude
for index, data in df.iterrows():
    lat_lng_coords = None
    print ('Now at PostalCode:', data['PostalCode'])
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(data['PostalCode']))
        lat_lng_coords = g.latlng
    data['Latitude'] = lat_lng_coords[0]
    data['Longitude'] = lat_lng_coords[1]
    print ('PostalCode:', data['PostalCode'], 'Latitude:', data['Latitude'], 'Longitude:', data['Longitude'])

In [ ]:

# updating these manually in case of geocoder exceeded for the day limit.
df1.loc[df1['Neighbourhood'] == 'Agincourt', ['Latitude','Longitude']] = 43.788009 ,-79.283882
df1.loc[df1['Neighbourhood'] == 'Alderwood', ['Latitude','Longitude']] = 43.6035413 ,-79.5464093
df1.loc[df1['Neighbourhood'] == 'Alexandra Park', ['Latitude','Longitude']] = 43.6497781 ,-79.4014566
df1.loc[df1['Neighbourhood'] == 'Allenby', ['Latitude','Longitude']] = 43.7077457 ,-79.41271429999999
df1.loc[df1['Neighbourhood'] == 'Amesbury', ['Latitude','Longitude']] = 43.7010934 ,-79.48100649999999
df1.loc[df1['Neighbourhood'] == 'Armour Heights', ['Latitude','Longitude']] = 43.7453882 ,-79.4225637
df1.loc[df1['Neighbourhood'] == 'Banbury', ['Latitude','Longitude']] = 43.7491152 ,-79.3663593
df1.loc[df1['Neighbourhood'] == 'Bathurst Manor', ['Latitude','Longitude']] = 43.7627405 ,-79.45632499999999
df1.loc[df1['Neighbourhood'] == 'Bay Street Corridor', ['Latitude','Longitude']] = 43.6567308 ,-79.3834603
df1.loc[df1['Neighbourhood'] == 'Bayview Village', ['Latitude','Longitude']] = 43.778243 ,-79.3827578
df1.loc[df1['Neighbourhood'] == 'Bayview Woods – Steeles', ['Latitude','Longitude']] = 43.7964111 ,-79.385975
df1.loc[df1['Neighbourhood'] == 'Bedford Park', ['Latitude','Longitude']] = 43.7301254 ,-79.4113072
df1.loc[df1['Neighbourhood'] == 'Bendale', ['Latitude','Longitude']] = 43.765434 ,-79.248935
df1.loc[df1['Neighbourhood'] == 'Birch Cliff', ['Latitude','Longitude']] = 43.6875 ,-79.269444
df1.loc[df1['Neighbourhood'] == 'Bloor West Village', ['Latitude','Longitude']] = 43.6508634 ,-79.477651
df1.loc[df1['Neighbourhood'] == 'Bracondale Hill', ['Latitude','Longitude']] = 43.678237 ,-79.4267845
df1.loc[df1['Neighbourhood'] == 'Branson', ['Latitude','Longitude']] = 43.7764213 ,-79.4506989
df1.loc[df1['Neighbourhood'] == 'Bridle Path', ['Latitude','Longitude']] = 43.7359565 ,-79.3692959
df1.loc[df1['Neighbourhood'] == 'Brockton', ['Latitude','Longitude']] = 43.6531909 ,-79.4380391
df1.loc[df1['Neighbourhood'] == 'Cabbagetown', ['Latitude','Longitude']] = 43.667263 ,-79.369169
df1.loc[df1['Neighbourhood'] == 'Caribou Park', ['Latitude','Longitude']] = 43.7182757 ,-79.4239707
df1.loc[df1['Neighbourhood'] == 'Carleton Village', ['Latitude','Longitude']] = 43.6716304 ,-79.4577314
df1.loc[df1['Neighbourhood'] == 'Casa Loma', ['Latitude','Longitude']] = 43.6780371 ,-79.4094439
df1.loc[df1['Neighbourhood'] == 'Centennial', ['Latitude','Longitude']] = 43.7841614 ,-79.1493545
df1.loc[df1['Neighbourhood'] == 'Chaplin Estates', ['Latitude','Longitude']] = 43.7019536 ,-79.4014566
df1.loc[df1['Neighbourhood'] == 'Christie Pits', ['Latitude','Longitude']] = 43.6645888 ,-79.4206809
df1.loc[df1['Neighbourhood'] == 'Church and Wellesley', ['Latitude','Longitude']] = 43.6645421 ,-79.3817523
df1.loc[df1['Neighbourhood'] == 'Clairville', ['Latitude','Longitude']] = 43.7425044 ,-79.62213679999999
df1.loc[df1['Neighbourhood'] == 'Clairlea', ['Latitude','Longitude']] = 43.7184961 ,-79.2914268
df1.loc[df1['Neighbourhood'] == 'Cliffcrest', ['Latitude','Longitude']] = 43.724917 ,-79.22632899999999
df1.loc[df1['Neighbourhood'] == 'Cliffside', ['Latitude','Longitude']] = 43.712217 ,-79.244001
df1.loc[df1['Neighbourhood'] == 'Corktown', ['Latitude','Longitude']] = 43.654787 ,-79.359932
df1.loc[df1['Neighbourhood'] == 'Crescent Town', ['Latitude','Longitude']] = 43.6959813 ,-79.2937357
df1.loc[df1['Neighbourhood'] == 'Cricket Club', ['Latitude','Longitude']] = 43.7385478 ,-79.41271429999999
df1.loc[df1['Neighbourhood'] == 'Davenport', ['Latitude','Longitude']] = 43.6719082 ,-79.4352256
df1.loc[df1['Neighbourhood'] == 'Davisville', ['Latitude','Longitude']] = 43.70430899999999 ,-79.388516
df1.loc[df1['Neighbourhood'] == 'Deer Park', ['Latitude','Longitude']] = 43.6898373 ,-79.398642
df1.loc[df1['Neighbourhood'] == 'Discovery District', ['Latitude','Longitude']] = 43.6603242 ,-79.3937162
df1.loc[df1['Neighbourhood'] == 'Distillery District/West Don Lands', ['Latitude','Longitude']] = 43.6529534 ,-79.3570685
df1.loc[df1['Neighbourhood'] == 'Don Mills', ['Latitude','Longitude']] = 43.7448473 ,-79.34092299999999
df1.loc[df1['Neighbourhood'] == 'Don Valley Village', ['Latitude','Longitude']] = 43.7837792 ,-79.352188
df1.loc[df1['Neighbourhood'] == 'Dorset Park', ['Latitude','Longitude']] = 43.7540011 ,-79.28211739999999
df1.loc[df1['Neighbourhood'] == 'Dovercourt Park', ['Latitude','Longitude']] = 43.6658469 ,-79.4338188
df1.loc[df1['Neighbourhood'] == 'Downsview', ['Latitude','Longitude']] = 43.732256 ,-79.49338499999999
df1.loc[df1['Neighbourhood'] == 'Dufferin Grove', ['Latitude','Longitude']] = 43.6558326 ,-79.4281914
df1.loc[df1['Neighbourhood'] == 'Earlscourt', ['Latitude','Longitude']] = 43.6769064 ,-79.4506989
df1.loc[df1['Neighbourhood'] == 'East Danforth', ['Latitude','Longitude']] = 43.6837565 ,-79.3212058
df1.loc[df1['Neighbourhood'] == 'Eatonville', ['Latitude','Longitude']] = 43.641448 ,-79.5547244
df1.loc[df1['Neighbourhood'] == 'Eglinton East', ['Latitude','Longitude']] = 43.7248346 ,-79.3051242
df1.loc[df1['Neighbourhood'] == 'Elia (Jane and Finch)', ['Latitude','Longitude']] = 43.7511187 ,-79.50975489999999
df1.loc[df1['Neighbourhood'] == 'Eringate', ['Latitude','Longitude']] = 43.66250369999999 ,-79.5772008
df1.loc[df1['Neighbourhood'] == 'Fairbank', ['Latitude','Longitude']] = 43.69640039999999 ,-79.45632499999999
df1.loc[df1['Neighbourhood'] == 'Fashion District', ['Latitude','Longitude']] = 43.644769 ,-79.398642
df1.loc[df1['Neighbourhood'] == 'Financial District', ['Latitude','Longitude']] = 43.64793479999999 ,-79.3817523
df1.loc[df1['Neighbourhood'] == 'Flemingdon Park', ['Latitude','Longitude']] = 43.720507 ,-79.33877799999999
df1.loc[df1['Neighbourhood'] == 'Forest Hill', ['Latitude','Longitude']] = 43.70431800000001 ,-79.4267845
df1.loc[df1['Neighbourhood'] == 'Fort York/Liberty Village', ['Latitude','Longitude']] = 43.6388363 ,-79.4003053
df1.loc[df1['Neighbourhood'] == 'Garden District', ['Latitude','Longitude']] = 43.656898 ,-79.37612179999999
df1.loc[df1['Neighbourhood'] == 'Glen Park', ['Latitude','Longitude']] = 43.7101005 ,-79.4506989
df1.loc[df1['Neighbourhood'] == 'Governor\'s Bridge/Bennington Heights', ['Latitude','Longitude']] = 43.693193 ,-79.37223809999999
df1.loc[df1['Neighbourhood'] == 'Grange Park', ['Latitude','Longitude']] = 43.6537341 ,-79.3930124
df1.loc[df1['Neighbourhood'] == 'Graydon Hall', ['Latitude','Longitude']] = 43.7616852 ,-79.34373939999999
df1.loc[df1['Neighbourhood'] == 'Guildwood', ['Latitude','Longitude']] = 43.752743 ,-79.19277699999999
df1.loc[df1['Neighbourhood'] == 'Harbord Village', ['Latitude','Longitude']] = 43.66190400000001 ,-79.4042712
df1.loc[df1['Neighbourhood'] == 'Harbourfront / CityPlace', ['Latitude','Longitude']] = 43.6416061 ,-79.3901975
df1.loc[df1['Neighbourhood'] == 'Harwood', ['Latitude','Longitude']] = 43.6758308 ,-79.4774195
df1.loc[df1['Neighbourhood'] == 'Henry Farm', ['Latitude','Longitude']] = 43.77178199999999 ,-79.352086
df1.loc[df1['Neighbourhood'] == 'High Park North', ['Latitude','Longitude']] = 43.6594968 ,-79.467576
df1.loc[df1['Neighbourhood'] == 'Highland Creek', ['Latitude','Longitude']] = 43.78658 ,-79.18829199999999
df1.loc[df1['Neighbourhood'] == 'Hillcrest', ['Latitude','Longitude']] = 43.678237 ,-79.4267845
df1.loc[df1['Neighbourhood'] == 'Hoggs Hollow', ['Latitude','Longitude']] = 43.7419701 ,-79.398642
df1.loc[df1['Neighbourhood'] == 'Humber Bay Shores', ['Latitude','Longitude']] = 43.6259939 ,-79.4774195
df1.loc[df1['Neighbourhood'] == 'Humber Heights', ['Latitude','Longitude']] = 43.6963578 ,-79.527033
df1.loc[df1['Neighbourhood'] == 'Humber Summit', ['Latitude','Longitude']] = 43.7601003 ,-79.5717846
df1.loc[df1['Neighbourhood'] == 'Humber Valley Village', ['Latitude','Longitude']] = 43.667113 ,-79.527974
df1.loc[df1['Neighbourhood'] == 'Humberlea', ['Latitude','Longitude']] = 43.721319 ,-79.5332169
df1.loc[df1['Neighbourhood'] == 'Humbermede', ['Latitude','Longitude']] = 43.7421301 ,-79.5406738
df1.loc[df1['Neighbourhood'] == 'Humberwood', ['Latitude','Longitude']] = 43.7256585 ,-79.619329
df1.loc[df1['Neighbourhood'] == 'Humewood–Cedarvale', ['Latitude','Longitude']] = 43.6937813 ,-79.4281914
df1.loc[df1['Neighbourhood'] == 'Ionview', ['Latitude','Longitude']] = 43.730824 ,-79.2739
df1.loc[df1['Neighbourhood'] == 'Islington – Six Points', ['Latitude','Longitude']] = 43.6467566 ,-79.53505299999999
df1.loc[df1['Neighbourhood'] == 'Junction Triangle', ['Latitude','Longitude']] = 43.6613564 ,-79.4492924
df1.loc[df1['Neighbourhood'] == 'Kensington Market', ['Latitude','Longitude']] = 43.6545236 ,-79.4014566
df1.loc[df1['Neighbourhood'] == 'Kingsview Village', ['Latitude','Longitude']] = 43.7005039 ,-79.5519144
df1.loc[df1['Neighbourhood'] == 'Lambton', ['Latitude','Longitude']] = 43.6673705 ,-79.5013207
df1.loc[df1['Neighbourhood'] == 'L\'Amoreaux', ['Latitude','Longitude']] = 43.796 ,-79.309
df1.loc[df1['Neighbourhood'] == 'Lansing', ['Latitude','Longitude']] = 43.7595964 ,-79.4225637
df1.loc[df1['Neighbourhood'] == 'Lawrence Heights', ['Latitude','Longitude']] = 43.72277400000001 ,-79.45092799999999
df1.loc[df1['Neighbourhood'] == 'Lawrence Manor', ['Latitude','Longitude']] = 43.7280111 ,-79.4394459
df1.loc[df1['Neighbourhood'] == 'Lawrence Park', ['Latitude','Longitude']] = 43.7232822 ,-79.3887901
df1.loc[df1['Neighbourhood'] == 'Leaside', ['Latitude','Longitude']] = 43.70882599999999 ,-79.373499
df1.loc[df1['Neighbourhood'] == 'Ledbury Park', ['Latitude','Longitude']] = 43.731176 ,-79.4225637
df1.loc[df1['Neighbourhood'] == 'Leslieville', ['Latitude','Longitude']] = 43.6658467 ,-79.33247329999999
df1.loc[df1['Neighbourhood'] == 'Little Italy', ['Latitude','Longitude']] = 43.6547828 ,-79.4169356
df1.loc[df1['Neighbourhood'] == 'Little Portugal', ['Latitude','Longitude']] = 43.6463408 ,-79.4281914
df1.loc[df1['Neighbourhood'] == 'Long Branch', ['Latitude','Longitude']] = 43.593421 ,-79.538164
df1.loc[df1['Neighbourhood'] == 'Lytton Park', ['Latitude','Longitude']] = 43.7159091 ,-79.4113072
df1.loc[df1['Neighbourhood'] == 'Malvern', ['Latitude','Longitude']] = 43.80916 ,-79.22169
df1.loc[df1['Neighbourhood'] == 'Maple Leaf', ['Latitude','Longitude']] = 43.7137562 ,-79.49007379999999
df1.loc[df1['Neighbourhood'] == 'Markland Wood', ['Latitude','Longitude']] = 43.632036 ,-79.57513999999999
df1.loc[df1['Neighbourhood'] == 'Maryvale', ['Latitude','Longitude']] = 43.762575 ,-79.314767
df1.loc[df1['Neighbourhood'] == 'Milliken', ['Latitude','Longitude']] = 43.8237331 ,-79.3070855
df1.loc[df1['Neighbourhood'] == 'Mimico', ['Latitude','Longitude']] = 43.614627 ,-79.4956974
df1.loc[df1['Neighbourhood'] == 'Moore Park', ['Latitude','Longitude']] = 43.6913089 ,-79.383686
df1.loc[df1['Neighbourhood'] == 'Morningside', ['Latitude','Longitude']] = 43.787 ,-79.206
df1.loc[df1['Neighbourhood'] == 'Mount Dennis', ['Latitude','Longitude']] = 43.687076 ,-79.48983299999999
df1.loc[df1['Neighbourhood'] == 'New Toronto', ['Latitude','Longitude']] = 43.6007776 ,-79.505257
df1.loc[df1['Neighbourhood'] == 'Newtonbrook', ['Latitude','Longitude']] = 43.790104 ,-79.4197497
df1.loc[df1['Neighbourhood'] == 'Niagara', ['Latitude','Longitude']] = 43.6373829 ,-79.40849279999999
df1.loc[df1['Neighbourhood'] == 'North York City Centre', ['Latitude','Longitude']] = 43.76830349999999 ,-79.4133325
df1.loc[df1['Neighbourhood'] == 'Oakridge', ['Latitude','Longitude']] = 43.696632 ,-79.282859
df1.loc[df1['Neighbourhood'] == 'O\'Connor–Parkview', ['Latitude','Longitude']] = 43.7063972 ,-79.30993699999999
df1.loc[df1['Neighbourhood'] == 'Old East York', ['Latitude','Longitude']] = 43.6964049 ,-79.3296565
df1.loc[df1['Neighbourhood'] == 'Old Mill/Baby Point', ['Latitude','Longitude']] = 43.6581547 ,-79.4914798
df1.loc[df1['Neighbourhood'] == 'Parkdale', ['Latitude','Longitude']] = 43.643832 ,-79.442534
df1.loc[df1['Neighbourhood'] == 'Parkway Forest', ['Latitude','Longitude']] = 43.7722041 ,-79.34233119999999
df1.loc[df1['Neighbourhood'] == 'Parkwoods', ['Latitude','Longitude']] = 43.7553609 ,-79.3268397
df1.loc[df1['Neighbourhood'] == 'Pelmo Park', ['Latitude','Longitude']] = 43.7121365 ,-79.5238103
df1.loc[df1['Neighbourhood'] == 'Playter Estates', ['Latitude','Longitude']] = 43.6790323 ,-79.359228
df1.loc[df1['Neighbourhood'] == 'Pleasant View', ['Latitude','Longitude']] = 43.7848218 ,-79.3381065
df1.loc[df1['Neighbourhood'] == 'Port Lands', ['Latitude','Longitude']] = 43.6473987 ,-79.3381065
df1.loc[df1['Neighbourhood'] == 'Port Union', ['Latitude','Longitude']] = 43.7834548 ,-79.149209
df1.loc[df1['Neighbourhood'] == 'Princess Gardens', ['Latitude','Longitude']] = 43.6688924 ,-79.5434841
df1.loc[df1['Neighbourhood'] == 'Regal Heights', ['Latitude','Longitude']] = 43.676652 ,-79.4352256
df1.loc[df1['Neighbourhood'] == 'Regent Park/Trefann Court', ['Latitude','Longitude']] = 43.6574889 ,-79.360284
df1.loc[df1['Neighbourhood'] == 'Richview', ['Latitude','Longitude']] = 43.6878699 ,-79.5434841
df1.loc[df1['Neighbourhood'] == 'Riverdale', ['Latitude','Longitude']] = 43.669714 ,-79.353185
df1.loc[df1['Neighbourhood'] == 'Rockcliffe–Smythe', ['Latitude','Longitude']] = 43.6742272 ,-79.49850909999999
df1.loc[df1['Neighbourhood'] == 'Roncesvalles', ['Latitude','Longitude']] = 43.6484365 ,-79.4506989
df1.loc[df1['Neighbourhood'] == 'Rosedale', ['Latitude','Longitude']] = 43.683375 ,-79.377172
df1.loc[df1['Neighbourhood'] == 'Rouge', ['Latitude','Longitude']] = 43.804929 ,-79.165842
df1.loc[df1['Neighbourhood'] == 'Rouge Hill', ['Latitude','Longitude']] = 43.794719 ,-79.134478
df1.loc[df1['Neighbourhood'] == 'Runnymede', ['Latitude','Longitude']] = 43.66173240000001 ,-79.4849101
df1.loc[df1['Neighbourhood'] == 'Scarborough Junction', ['Latitude','Longitude']] = 43.71666700000001 ,-79.259722
df1.loc[df1['Neighbourhood'] == 'Scarborough City Centre', ['Latitude','Longitude']] = 43.7762792 ,-79.2529857
df1.loc[df1['Neighbourhood'] == 'Scarborough Village', ['Latitude','Longitude']] = 43.740615 ,-79.208887
df1.loc[df1['Neighbourhood'] == 'Seaton Village', ['Latitude','Longitude']] = 43.669017 ,-79.4169356
df1.loc[df1['Neighbourhood'] == 'Silverthorn', ['Latitude','Longitude']] = 43.6879613 ,-79.467576
df1.loc[df1['Neighbourhood'] == 'Smithfield', ['Latitude','Longitude']] = 43.7394164 ,-79.58843689999999
df1.loc[df1['Neighbourhood'] == 'South Hill', ['Latitude','Longitude']] = 43.6821959 ,-79.4056784
df1.loc[df1['Neighbourhood'] == 'St. James Town', ['Latitude','Longitude']] = 43.6708666 ,-79.37330639999999
df1.loc[df1['Neighbourhood'] == 'Steeles', ['Latitude','Longitude']] = 43.8155519 ,-79.3107689
df1.loc[df1['Neighbourhood'] == 'Summerhill', ['Latitude','Longitude']] = 43.6811436 ,-79.3944199
df1.loc[df1['Neighbourhood'] == 'Sunnylea', ['Latitude','Longitude']] = 43.6399324 ,-79.5125662
df1.loc[df1['Neighbourhood'] == 'Swansea', ['Latitude','Longitude']] = 43.6439713 ,-79.47799069999999
df1.loc[df1['Neighbourhood'] == 'Tam O\'Shanter – Sullivan', ['Latitude','Longitude']] = 43.788056 ,-79.301389
df1.loc[df1['Neighbourhood'] == 'The Annex', ['Latitude','Longitude']] = 43.6698329 ,-79.407585
df1.loc[df1['Neighbourhood'] == 'The Beaches', ['Latitude','Longitude']] = 43.6673479 ,-79.29669299999999
df1.loc[df1['Neighbourhood'] == 'The Danforth', ['Latitude','Longitude']] = 43.6837565 ,-79.3212058
df1.loc[df1['Neighbourhood'] == 'The Elms', ['Latitude','Longitude']] = 43.7236919 ,-79.5462943
df1.loc[df1['Neighbourhood'] == 'The Junction', ['Latitude','Longitude']] = 43.666612 ,-79.468614
df1.loc[df1['Neighbourhood'] == 'The Kingsway', ['Latitude','Longitude']] = 43.652674 ,-79.507194
df1.loc[df1['Neighbourhood'] == 'The Queensway', ['Latitude','Longitude']] = 43.6254023 ,-79.5068881
df1.loc[df1['Neighbourhood'] == 'Thistletown', ['Latitude','Longitude']] = 43.73717449999999 ,-79.5652707
df1.loc[df1['Neighbourhood'] == 'Thorncliffe Park', ['Latitude','Longitude']] = 43.7080032 ,-79.352188
df1.loc[df1['Neighbourhood'] == 'Toronto Islands', ['Latitude','Longitude']] = 43.621413 ,-79.3788412
df1.loc[df1['Neighbourhood'] == 'Trinity–Bellwoods', ['Latitude','Longitude']] = 43.6500371 ,-79.4169356
df1.loc[df1['Neighbourhood'] == 'Upper Beaches', ['Latitude','Longitude']] = 43.6779526 ,-79.30993699999999
df1.loc[df1['Neighbourhood'] == 'Victoria Village', ['Latitude','Longitude']] = 43.7357352 ,-79.3124184
df1.loc[df1['Neighbourhood'] == 'Wallace Emerson', ['Latitude','Longitude']] = 43.6684011 ,-79.4405951
df1.loc[df1['Neighbourhood'] == 'West Deane Park', ['Latitude','Longitude']] = 43.6635853 ,-79.5631537
df1.loc[df1['Neighbourhood'] == 'West Hill', ['Latitude','Longitude']] = 43.769163 ,-79.182803
df1.loc[df1['Neighbourhood'] == 'West Rouge', ['Latitude','Longitude']] = 43.794719 ,-79.134478
df1.loc[df1['Neighbourhood'] == 'Westminster', ['Latitude','Longitude']] = 43.7764213 ,-79.4506989
df1.loc[df1['Neighbourhood'] == 'Westmount', ['Latitude','Longitude']] = 43.6871106 ,-79.52240479999999
df1.loc[df1['Neighbourhood'] == 'Weston', ['Latitude','Longitude']] = 43.7002318 ,-79.51507810000001
df1.loc[df1['Neighbourhood'] == 'Wexford', ['Latitude','Longitude']] = 43.74320789999999 ,-79.3046444
df1.loc[df1['Neighbourhood'] == 'Willowdale', ['Latitude','Longitude']] = 43.7711708 ,-79.4197497
df1.loc[df1['Neighbourhood'] == 'Wilson Heights', ['Latitude','Longitude']] = 43.74069799999999 ,-79.438824
df1.loc[df1['Neighbourhood'] == 'Woburn', ['Latitude','Longitude']] = 43.767528 ,-79.21772100000001
df1.loc[df1['Neighbourhood'] == 'Wychwood', ['Latitude','Longitude']] = 43.6779102 ,-79.4201015
df1.loc[df1['Neighbourhood'] == 'York Mills', ['Latitude','Longitude']] = 43.7501272 ,-79.39723459999999
df1.loc[df1['Neighbourhood'] == 'York University Heights', ['Latitude','Longitude']] = 43.7679803 ,-79.4872619
df1.loc[df1['Neighbourhood'] == 'Yorkville', ['Latitude','Longitude']] = 43.670749 ,-79.39304

In [ ]:
# print out the updated dataframe
df1.head(10)

In [ ]:
# double check on the dataframe
print('The dataframe has {} neighborhoods.'.format(
        len(df1['Neighbourhood'].unique()),
        df1.shape[0]
    )
)

In [ ]:
total_population = df1['Population'].sum()
#print (total_population)

# the higher populated neighbourhood will have the higher score
df1['Population Score'] = df1['Population']/total_population*100

# sort the dataframe based on the descending population score
df1 = df1.sort_values('Population Score', ascending=False)
df1.head(10)

In [ ]:
population_chart = df1[['Neighbourhood', 'Population']].copy()
top_chart = population_chart.head(10)
top_chart.set_index('Neighbourhood', inplace=True)

# plot data
colors = ['#5cb85c', '#5bc0de', '#d9534f']
top_chart.plot(kind='bar', figsize=(12, 8), width=0.6, fontsize=14, color=colors)

plt.xlabel('Neighbourhood') # add to x-label to the plot
plt.ylabel('Population') # add y-label to the plot
plt.title('Top 10 Populated Neighbourhoods in Toronto') # add title to the plot

plt.show()

In [ ]:

# Use geopy library to get the latitude and longitude values of Toronto Ontario
address = 'Toronto Ontario, TO'
toronto_data = df1
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Ontario are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Toronto Ontario using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  

map_to

In [ ]:
# Foursquare Credentials
CLIENT_ID = 'RF2HLDBFJ0Z0IAOHHOTQG4CMUCDM0RGVAF3DVCQFD153FMOB' # your Foursquare ID
CLIENT_SECRET = '2LYBEQRR5TXMPQWUFKMLWCIKWVOA2G3U3Y1YCYL20MWXKRHJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:

# explore the first neighborhood in our dataframe.
toronto_data.loc[1, 'Neighbourhood']
toronto_data.head(1)

In [ ]:
neighborhood_latitude = toronto_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[1, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [ ]:
# explore the top 100 venues
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

In [ ]:

results = requests.get(url).json()
results

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
# clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


In [ ]:

# Explore Neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:

# get scarborough venues
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

print(toronto_venues.shape)
toronto_venues.head(10)

In [ ]:

# find out how many unique categories
toronto_venues.groupby('Neighbourhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


In [ ]:
# analyze each neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape


In [ ]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head(10)